## Overview

-   Build a *simple* classifier to predict the distribution of a species

-   No, I will not tell you which species, it's a large North American mammal

-   Use this as an opportunity to talk about interpretable ML

-   Discuss which biases are appropriate in a predictive model

::: footer
CC BY 4.0 - Timothée Poisot
:::

------------------------------------------------------------------------

::: r-fit-text
We care a lot about the

**process**

and only a little about the

**product**
:::

------------------------------------------------------------------------

## Why...

... think of SDMs as a ML problem?

:   They are (they really, really are, see @beery2021)

... think of explainable ML for SDM?

:   Uptake of models *requires* transparent predictions

... not tell us which species this is about?

:   Because this is the point (you'll see)

## Do try this at home!

💻 + 📔 + 🗺️ at `https://github.com/tpoisot/InterpretableSDMWithJulia/`


In [ ]:
#| label: Include the packages we need
#| echo: true
#| output: false
include(joinpath("code", "pkg.jl")); # Dependencies
include(joinpath("code", "nbc.jl")); # Naive Bayes Classifier
include(joinpath("code", "bioclim.jl")); # BioClim model
include(joinpath("code", "confusion.jl")); # Confusion matrix utilities
include(joinpath("code", "splitters.jl")); # Cross-validation (part one)
include(joinpath("code", "crossvalidate.jl")); # Cross-validation (part deux)
include(joinpath("code", "variableselection.jl")); # Variable selection
include(joinpath("code", "shapley.jl")); # Shapley values
include(joinpath("code", "palettes.jl")); # Accessible color palettes

## Species occurrences


In [ ]:
#| label: Get the species data
#| eval: true
#| echo: true
#| output: false
sightings = CSV.File("occurrences.csv")
occ = [
    (record.longitude, record.latitude)
    for record in sightings
    if record.classification == "Class A"
]
filter!(r -> -90 <= r[2] <= 90, occ)
filter!(r -> -180 <= r[1] <= 180, occ)
boundingbox = (
    left = minimum(first.(occ)),
    right = maximum(first.(occ)),
    bottom = minimum(last.(occ)),
    top = maximum(last.(occ)),
)

## Bioclimatic data

We collect BioClim data from CHELSA v1, using `SpeciesDistributionToolkit`


In [ ]:
#| label: Download the BioClim data from WorldClim2
#| eval: true
#| echo: true
#| output: false
provider = RasterData(WorldClim2, BioClim)
opts = (; resolution=5.0)
temperature = SimpleSDMPredictor(provider, layer=1; opts..., boundingbox...)

::: footer
BioClim data from @karger2020; see @dansereau2021 for more about the packages
:::

## Bioclimatic data

We set the pixels with only open water to `nothing`


In [ ]:
#| label: Get the open water pixels
#| eval: true
#| output: false
#| echo: true
water = 
    SimpleSDMPredictor(RasterData(EarthEnv, LandCover), layer=12; boundingbox...)
land = similar(temperature, Bool)
replace!(land, false => true)
for k in keys(land)
    if !isnothing(water[k])
        if water[k] == 100
            land[k] = false
        end
    end
end
temperature = mask(land, temperature)

::: footer
Land-cover data from @tuanmu2014
:::

## Where are we so far?


In [ ]:
#| echo: false
fig = Figure(; resolution=(900, 500))
ax = Axis(fig[1,1]; xlabel="Longitude", ylabel="Latitude", aspect=DataAspect())
hm = heatmap!(ax, temperature, colormap=[:lightgrey])
Colorbar(fig[1,2], hm; tellheight=false)
scatter!(ax, occ; color=:black, marker=:cross, markersize=3)
current_figure()

## Spatial thinning

We limit the occurrences to one per grid cell, assigned to the center of the grid cell


In [ ]:
#| label: Make the layer for presences
#| echo: true
#| eval: true
#| output: false
presence_layer = similar(temperature, Bool)
for i in axes(occ, 1)
    if ~isnothing(presence_layer[occ[i]...])
        presence_layer[occ[i]...] = true
    end
end

## Background points generation

We generate background points proportionally to the distance away from observations, with a 10km buffer around each point with no background point allowed:


In [ ]:
#| label: Make the pseudo-absence buffer
#| eval: true
#| echo: true
#| output: false
possible_background = pseudoabsencemask(DistanceToEvent, presence_layer)

And then we sample three pseudo-absence for each occurrence:


In [ ]:
#| label: Make the absence layer
#| echo: true
#| eval: true
#| output: false
absence_layer = backgroundpoints(
    (x -> x^1.01).(possible_background), 
    3sum(presence_layer);
    replace=false
)

::: footer
See @barbet-massin2012 for more on background points
:::

## Background points cleaning

We can remove all of the information that is neither a presence nor a pseudo-absence


In [ ]:
#| label: Pseudo-absence/presence remove
#| output: false
#| echo: true
replace!(absence_layer, false => nothing)
replace!(presence_layer, false => nothing)

## Data overview


In [ ]:
scatter!(keys(absence_layer); color = :red, marker=:xcross, markersize=2)
current_figure()

In [ ]:
#| label: Save the stack of layers
#| echo: false
#| output: false
#| eval: true
predictors = [
    mask(land, SimpleSDMPredictor(provider; layer = l, opts..., boundingbox...))
    for l in layers(provider)
]

## Preparing the responses and variables


In [ ]:
#| label: Assemble y and X
#| echo: true
#| output: false
Xpresence = hcat([bioclim_var[keys(presence_layer)] for bioclim_var in predictors]...)
ypresence = fill(true, length(presence_layer))
Xabsence = hcat([bioclim_var[keys(absence_layer)] for bioclim_var in predictors]...)
yabsence = fill(false, length(absence_layer))
X = vcat(Xpresence, Xabsence)
y = vcat(ypresence, yabsence)

In [ ]:
#| output: false
#| echo: false
#| eval: true
bclay = layers(RasterData(WorldClim2, BioClim))
bcdes = layerdescriptions(RasterData(WorldClim2, BioClim))
presences = Tuple.(keys(presence_layer))
absences = Tuple.(keys(absence_layer))
variables = [(bc, var_trim(bcdes[bc])) for bc in bclay]

## The model -- Naive Bayes Classifier

Prediction:

$$
P(+|x) = \frac{P(+)}{P(x)}P(x|+)
$$

Decision rule:

$$
\hat y = \text{argmax}_j \, P(\mathbf{c}_j)\prod_i P(\mathbf{x}_i|\mathbf{c}_j)
$$

::: footer
With $n$ instances and $f$ features, NBC trains *and* predicts in $\mathcal{O}(n\times f)$
:::

## The model -- Naive Bayes Classifier

Assumption of Gaussian distributions:

$$
P(x|+) = \text{pdf}(x, \mathcal{N}(\mu_+, \sigma_+))
$$

## Cross-validation

We keep an **unseen** *testing* set -- this will be used at the very end to report expected model performance


In [ ]:
#| label: Testing set
#| echo: true
#| output: false
idx, tidx = holdout(y, X; permute=true)

For *validation*, we will run k-folds


In [ ]:
#| label: k-folds
#| echo: true
#| output: false
ty, tX = y[idx], X[idx,:]
folds = kfold(ty, tX; k=15, permute=true)
k = length(folds)

::: footer
See @valavi2018 for more on cross-validation
:::

## A note on cross-validation

All models share the same folds

:   we can compare the validation performance across experiments to select the best model

Model performance can be compared

:   we average the relevant summary statistics over each validation set

Testing set is *only* for future evaluation

:   we can only use it once and report the expected performance *of the best model*

## Baseline performance

We need to get a sense of how difficult the classification problem is:


In [ ]:
#| echo: true
#| output: false
N_v0 = crossvalidate(naivebayes, ty, tX, folds)
B_v0 = crossvalidate(bioclim, ty, tX, folds, eps())

This uses an un-tuned model with all variables and reports the average over all validation sets. In addition, we will always use the BioClim model as a comparison.

## Measures on the confusion matrix {.smaller}

|     | BioClim                       | NBC                           |
|-----|-------------------------------|-------------------------------|
| FPR | `{julia} sm(fpr, B_v0)`       | `{julia} sm(fpr, N_v0)`       |
| FNR | `{julia} sm(fnr, B_v0)`       | `{julia} sm(fnr, N_v0)`       |
| TPR | `{julia} sm(tpr, B_v0)`       | `{julia} sm(tpr, N_v0)`       |
| TNR | `{julia} sm(tnr, B_v0)`       | `{julia} sm(tnr, N_v0)`       |
| TSS | `{julia} sm(trueskill, B_v0)` | `{julia} sm(trueskill, N_v0)` |
| MCC | `{julia} sm(mcc, B_v0)`       | `{julia} sm(mcc, N_v0)`       |

::: footer
It's a good idea to check the values for the training sets too...
:::

## Variable selection

We add variables one at a time, until the Matthew's Correlation Coefficient stops increasing -- we keep annual temperature, isothermality, mean diurnal range, and annual precipitation


In [ ]:
#| echo: true
#| output: false
available_variables = forwardselection(ty, tX, folds, naivebayes, mcc)

This method identifies `{julia} length(available_variables)` variables, some of which are:

1.  `{julia} variables[available_variables[1]][2]`

2.  `{julia} variables[available_variables[2]][2]`

3.  `{julia} variables[available_variables[3]][2]`

## Variable selection?

-   Constrained variable selection

-   VIF threshold (over the extent or over document occurrences?)

-   PCA for dimensionality reduction *v.* Whitening for colinearity removal

-   Potential for data leakage: data transformations don't exist, they are just models we can train

## Model with variable selection


In [ ]:
#| echo: true
#| output: false
N_v1 = crossvalidate(naivebayes, ty, tX[:,available_variables], folds)
B_v1 = crossvalidate(bioclim, ty, tX[:,available_variables], folds, eps())

## Measures on the confusion matrix {.smaller}

|     | BioClim                       | NBC                           | BioClim (v.s.)                | NBC (v.s.)                    |
|---------------|---------------|---------------|---------------|---------------|
| FPR | `{julia} sm(fpr, B_v0)`       | `{julia} sm(fpr, N_v0)`       | `{julia} sm(fpr, B_v1)`       | `{julia} sm(fpr, N_v1)`       |
| FNR | `{julia} sm(fnr, B_v0)`       | `{julia} sm(fnr, N_v0)`       | `{julia} sm(fnr, B_v1)`       | `{julia} sm(fnr, N_v1)`       |
| TPR | `{julia} sm(tpr, B_v0)`       | `{julia} sm(tpr, N_v0)`       | `{julia} sm(tpr, B_v1)`       | `{julia} sm(tpr, N_v1)`       |
| TNR | `{julia} sm(tnr, B_v0)`       | `{julia} sm(tnr, N_v0)`       | `{julia} sm(tnr, B_v1)`       | `{julia} sm(tnr, N_v1)`       |
| TSS | `{julia} sm(trueskill, B_v0)` | `{julia} sm(trueskill, N_v0)` | `{julia} sm(trueskill, B_v1)` | `{julia} sm(trueskill, N_v1)` |
| MCC | `{julia} sm(mcc, B_v0)`       | `{julia} sm(mcc, N_v0)`       | `{julia} sm(mcc, B_v1)`       | `{julia} sm(mcc, N_v1)`       |

## How do we make the model better?

The NBC is a *probabilistic classifier* returning $P(+|\mathbf{x})$

The *decision rule* is to assign a presence when $P(\cdot) > 0.5$

But $P(\cdot) > \tau$ is a far more general approach, and we can use learning curves to identify $\tau$

## Thresholding the model


In [ ]:
#| echo: true
#| output: false
thr = LinRange(0.0, 1.0, 500)
T = hcat([crossvalidate(naivebayes, ty, tX[:,available_variables], folds, t) for t in thr]...)

## But how do we pick the threshold?


In [ ]:
#| echo: false
fig = Figure(; resolution=(900, 450))

g1 = GridLayout(fig[1:2,1])

xt = 0.0:0.2:1.0
yt = 0.0:0.25:1.0

ax_mcc = Axis(g1[1,1], ylabel="MCC"; yticks=yt, xticks=xt)
ax_fpr = Axis(g1[2,1], ylabel="False Pos. Rate"; yaxisposition=:right, yticks=yt, xticks=xt)
ax_fnr = Axis(g1[3,1], xlabel="Threshold", ylabel="False Neg. Rate", yticks=yt, xticks=xt)

hidexdecorations!(ax_fpr, grid=false)
hidexdecorations!(ax_mcc, grid=false)
rowgap!(g1, 6)

scores = mcc.(T)
μ = vec(mean(scores; dims=1))

for i in axes(T, 1)
    lines!(ax_mcc, thr, mcc.(T[i,:]), color=:grey, alpha=0.6)
    lines!(ax_fpr, thr, fpr.(T[i,:]), color=:grey, alpha=0.6)
    lines!(ax_fnr, thr, fnr.(T[i,:]), color=:grey, alpha=0.6)
end

mmax, m = findmax(μ)
scatter!(ax_mcc, thr[m], mmax, color=:red, markersize=10)
rangebars!(ax_mcc, [thr[m]], 0.98*[minimum(scores[:,m])], 1.02*[maximum(scores[:,m])], color=:red, whiskerwidth=10)

xlims!(ax_mcc, low=0.0, high=1.0)
ylims!(ax_mcc, low=0.0, high=1.0)
xlims!(ax_fpr, low=0.0, high=1.0)
ylims!(ax_fpr, low=0.0, high=1.0)
xlims!(ax_fnr, low=0.0, high=1.0)
ylims!(ax_fnr, low=0.0, high=1.0)

ax_roc = Axis(fig[1,2], xlabel="False Positive Rate", ylabel="True Positive Rate", aspect=1)
ax_pr = Axis(fig[2,2], xlabel="True Positive Rate", ylabel="Positive Pred. Value", aspect=1)

for i in axes(T, 1)
    scatter!(ax_roc, fpr.(T[i,:]), tpr.(T[i,:]), color=:grey, markersize=2, alpha=0.6)
    scatter!(ax_pr, tpr.(T[i,:]), ppv.(T[i,:]), color=:grey, markersize=2, alpha=0.6)
end

arrows!(ax_roc, [0.5], [0.5], [-0.2], [0.2], color=:blue, linewidth=4)
arrows!(ax_roc, [0.5], [0.5], [0.2], [-0.2], color=:red, linewidth=2, linestyle=:dash)

lines!(ax_roc, [0.0, 1.0], [0.0, 1.0], color=:black, linestyle=:dash)
lines!(ax_pr, [0.0, 1.0], [0.5, 0.5], color=:black, linestyle=:dash)

arrows!(ax_pr, [0.5], [0.5], [0.2], [0.2], color=:blue, linewidth=4)
arrows!(ax_pr, [0.5], [0.5], [-0.2], [-0.2], color=:red, linewidth=2, linestyle=:dash)

xlims!(ax_roc, low=0.0, high=1.0)
ylims!(ax_roc, low=0.0, high=1.0)
xlims!(ax_pr, low=0.0, high=1.0)
ylims!(ax_pr, low=0.0, high=1.0)

current_figure()

## Tuned model with selected variables


In [ ]:
#| echo: true
#| output: false
N_v2 = crossvalidate(naivebayes, ty, tX[:,available_variables], folds, thr[m])

## Measures on the confusion matrix {.smaller}

|     | BioClim                       | NBC                           | BioClim (v.s.)                | NBC (v.s.)                    | NBC (v.s. + tuning)           |
|------------|------------|------------|------------|------------|------------|
| FPR | `{julia} sm(fpr, B_v0)`       | `{julia} sm(fpr, N_v0)`       | `{julia} sm(fpr, B_v1)`       | `{julia} sm(fpr, N_v1)`       | `{julia} sm(fpr, N_v2)`       |
| FNR | `{julia} sm(fnr, B_v0)`       | `{julia} sm(fnr, N_v0)`       | `{julia} sm(fnr, B_v1)`       | `{julia} sm(fnr, N_v1)`       | `{julia} sm(fnr, N_v2)`       |
| TPR | `{julia} sm(tpr, B_v0)`       | `{julia} sm(tpr, N_v0)`       | `{julia} sm(tpr, B_v1)`       | `{julia} sm(tpr, N_v1)`       | `{julia} sm(tpr, N_v2)`       |
| TNR | `{julia} sm(tnr, B_v0)`       | `{julia} sm(tnr, N_v0)`       | `{julia} sm(tnr, B_v1)`       | `{julia} sm(tnr, N_v1)`       | `{julia} sm(tnr, N_v2)`       |
| TSS | `{julia} sm(trueskill, B_v0)` | `{julia} sm(trueskill, N_v0)` | `{julia} sm(trueskill, B_v1)` | `{julia} sm(trueskill, N_v1)` | `{julia} sm(trueskill, N_v2)` |
| MCC | `{julia} sm(mcc, B_v0)`       | `{julia} sm(mcc, N_v0)`       | `{julia} sm(mcc, B_v1)`       | `{julia} sm(mcc, N_v1)`       | `{julia} sm(mcc, N_v2)`       |

## How do we make the model better?

The NBC is a *Bayesian classifier* returning $P(+|\mathbf{x})$

The *actual probability* depends on $P(+)$

There is no reason not to also tune $P(+)$ (jointly with other hyper-parameters)!

## Joint tuning of hyper-parameters


In [ ]:
#| echo: true
#| output: false
thr = LinRange(0.0, 1.0, 50)
pplus = LinRange(0.0, 1.0, 50)
T = [crossvalidate(naivebayes, ty, tX[:,available_variables], folds, t; presence=prior) for t in thr, prior in pplus]
best_mcc, params = findmax(map(v -> mean(mcc.(v)), T))
τ = thr[params.I[1]]
ppres = pplus[params.I[2]]

## Tuned (again) model with selected variables


In [ ]:
#| echo: true
#| output: false
N_v3 = crossvalidate(naivebayes, ty, tX[:,available_variables], folds, τ; presence=ppres)

## Measures on the confusion matrix {.smaller}

|     | BioClim                       | NBC (v0)                      | NBC (v1)                      | NBC (v2)                      | NBC (v3)                      |
|------------|------------|-------------|------------|------------|------------|
| FPR | `{julia} sm(fpr, B_v0)`       | `{julia} sm(fpr, N_v0)`       | `{julia} sm(fpr, N_v1)`       | `{julia} sm(fpr, N_v2)`       | `{julia} sm(fpr, N_v3)`       |
| FNR | `{julia} sm(fnr, B_v0)`       | `{julia} sm(fnr, N_v0)`       | `{julia} sm(fnr, N_v1)`       | `{julia} sm(fnr, N_v2)`       | `{julia} sm(fnr, N_v3)`       |
| TPR | `{julia} sm(tpr, B_v0)`       | `{julia} sm(tpr, N_v0)`       | `{julia} sm(tpr, N_v1)`       | `{julia} sm(tpr, N_v2)`       | `{julia} sm(tpr, N_v3)`       |
| TNR | `{julia} sm(tnr, B_v0)`       | `{julia} sm(tnr, N_v0)`       | `{julia} sm(tnr, N_v1)`       | `{julia} sm(tnr, N_v2)`       | `{julia} sm(tnr, N_v3)`       |
| TSS | `{julia} sm(trueskill, B_v0)` | `{julia} sm(trueskill, N_v0)` | `{julia} sm(trueskill, N_v1)` | `{julia} sm(trueskill, N_v2)` | `{julia} sm(trueskill, N_v3)` |
| MCC | `{julia} sm(mcc, B_v0)`       | `{julia} sm(mcc, N_v0)`       | `{julia} sm(mcc, N_v1)`       | `{julia} sm(mcc, N_v2)`       | `{julia} sm(mcc, N_v3)`       |

## Tuned model performance

We can retrain over *all* the training data


In [ ]:
#| echo: true
#| output: false
finalmodel = naivebayes(ty, tX[:,available_variables]; presence=ppres)
prediction = vec(mapslices(finalmodel, X[tidx,available_variables]; dims=2))
C = ConfusionMatrix(prediction, y[tidx], τ)

## Estimated performance

|     | Final model                |
|-----|----------------------------|
| FPR | `{julia} sm(fpr, C)`       |
| FNR | `{julia} sm(fnr, C)`       |
| TPR | `{julia} sm(tpr, C)`       |
| TNR | `{julia} sm(tnr, C)`       |
| TSS | `{julia} sm(trueskill, C)` |
| MCC | `{julia} sm(mcc, C)`       |

## Acceptable bias

-   false positives: we expect that our knowledge of the distribution is incomplete, and *this is why we train a model*

-   false negatives: the detection of the species is imperfect (we will reveal the species soon)

## Prediction for each pixel


In [ ]:
#| echo: false
#| eval: true
#| output: false
samplemodels = [naivebayes(ty, tX[b,available_variables]; presence=ppres) for b in bootstrap(ty, tX)]

In [ ]:
#| echo: true
#| output: false
prediction = similar(temperature, Float64)
variability = similar(temperature, Float64)
uncertainty = similar(temperature, Float64)
Threads.@threads for k in keys(prediction)
    pred_k = [p[k] for p in predictors[available_variables]]
    bootstraps = [
            samplemodel(pred_k)
            for samplemodel in samplemodels
        ]
    prediction[k] = finalmodel(pred_k)
    variability[k] = iqr(bootstraps)
    uncertainty[k] = entropy(prediction[k])
end

## Tuned model - prediction


In [ ]:
#| echo: false
#| output: true
fig = Figure(; resolution=(900, 500))
ax = Axis(fig[1,1]; xlabel="Longitude", ylabel="Latitude", aspect=DataAspect())
hm = heatmap!(ax, prediction, colormap=cgrad(iridescent), colorrange=(0., 1.))
Colorbar(fig[1,2], hm; tellheight=false)
current_figure()

## Tuned model - variability in output


In [ ]:
#| echo: false
#| output: true
fig = Figure(; resolution=(900, 500))
ax = Axis(fig[1,1]; xlabel="Longitude", ylabel="Latitude", aspect=DataAspect())
hm = heatmap!(ax, variability, colormap=cgrad(iridescent), colorrange=extrema(variability))
Colorbar(fig[1,2], hm; tellheight=false)
current_figure()

::: footer
IQR for 50 bootstrap replicates
:::

## Tuned model - entropy in probability


In [ ]:
#| echo: false
#| output: true
fig = Figure(; resolution=(900, 500))
ax = Axis(fig[1,1]; xlabel="Longitude", ylabel="Latitude", aspect=DataAspect())
hm = heatmap!(ax, uncertainty, colormap=cgrad(iridescent), colorrange=(0., 1.))
Colorbar(fig[1,2], hm; tellheight=false)
current_figure()

::: footer
Entropy (in bits) of the NBC probability
:::

## Tuned model - range


In [ ]:
#| echo: false
#| output: true
fig = Figure(; resolution=(900, 500))
ax = Axis(fig[1,1]; xlabel="Longitude", ylabel="Latitude", aspect=DataAspect())
hm = heatmap!(prediction .>= τ; colormap = cgrad([:lightgrey, :black]; alpha = 0.3))
Colorbar(fig[1,2], hm; tellheight=false)
current_figure()

::: footer
Probability \> `{julia} round(thr[m]; digits=3)`
:::

## Predicting the predictions?

Shapley values (Monte-Carlo approximation): if we mix the variables across two observations, how important is the $i$-th variable?

Expresses "importance" as an additive factor on top of the *average* prediction (here: average prob. of occurrence)


In [ ]:
#| echo: true
#| output: false
shapval = [similar(first(predictors), Float64) for i in eachindex(available_variables)]
Threads.@threads for k in keys(shapval[1])
    x = [p[k] for p in predictors[available_variables]]
    for i in axes(shapval, 1)
        shapval[i][k] = shapleyvalues(finalmodel, tX[:,available_variables], x, i; M=50)
        if isnan(shapval[i][k])
            shapval[i][k] = 0.0
        end
    end
end

## Importance of variables


In [ ]:
#| echo: true
varimp = sum.(map(abs, shapval))
varimp ./= sum(varimp)
for v in sortperm(varimp, rev=true)
    vname = variables[available_variables[v]][2]
    vctr = round(Int, varimp[v]*100)
    println("$(vname) - $(vctr)%")
end

There is a difference between **contributing to model performance** and **contributing to model explainability**

## Top three variables


In [ ]:
#| echo: false
#| output: true
fig = Figure(; resolution=(1500, 900))

totalvar = sum(map(abs, shapval))

gl = fig[1,1] = GridLayout()

fpos = 1
for i in sortperm(varimp; rev=true)[1:3]
    ax_mp = Axis(gl[fpos,1], aspect=DataAspect()) # Absolute contribution
    scl = maximum(abs.(extrema(shapval[i]))).*(-1,1)
    heatmap!(ax_mp, shapval[i], colorrange=scl, colormap=cgrad(nightfall), aspect=DataAspect())
    hidexdecorations!(ax_mp)
    hideydecorations!(ax_mp)

    ax_pp = Axis(gl[fpos,2], aspect=DataAspect()) # Relative contribution
    relcon = abs(shapval[i])/totalvar
    heatmap!(ax_pp, relcon, colorrange=(0, 1), colormap=cgrad(iridescent), aspect=DataAspect())
    hidexdecorations!(ax_pp)
    hideydecorations!(ax_pp)

    ax_pr = Axis(gl[fpos,3], title=variables[available_variables[i]][2]) # Partial response
    ylims!(ax_pr, scl)
    hexbin!(ax_pr, predictors[available_variables[i]], shapval[i], bins=200, colormap=cgrad(sunset))
    fpos += 1
end

current_figure()

## Most determinant predictor


In [ ]:
shapmax = mosaic(argmax, map(abs, shapval[sortperm(varimp; rev=true)]))
for k in keys(shapmax)
    if shapmax[k] > 3
        shapmax[k] = 4
    end
end
pal = cgrad([colorant"#ddaa33", colorant"#bb5566", colorant"#004488", colorant"#000000"], 4, categorical=true)
fig = Figure(; resolution=(900, 500))
ax = Axis(fig[1,1]; xlabel="Longitude", ylabel="Latitude", aspect=DataAspect())
hm = heatmap!(ax, shapmax, colormap=pal)
Colorbar(fig[1,2], hm; tellheight=false)
current_figure()

## But wait...

> What do you think the species was?

## Take-home

-   building a model is *incremental*

-   each step adds arbitrary decisions we can control for, justify, or live with

-   we can provide explanations for every single prediction

-   free online textbook (in development) at `https://tpoisot.github.io/DataSciForBiodivSci/`

## References